In [2]:
from pathlib import Path
import polars as pl
from datasets import Dataset, concatenate_datasets, load_from_disk
import numpy as np
from autofaiss import build_index
from src.item_matching.build_index.model import Model

/home/kevin/PycharmProjects/item_matching/src/item_matching/build_index/func.py:8: SyntaxWarning: invalid escape sequence '\('
  regex = "[\(\[\<\"].*?[\)\]\>\"]"


In [5]:
path = Path.home() / 'Downloads/cb'
path_db = path / 'cb_2024-03-07.parquet'

col = ['item_id', 'item_name']
df_db = (
    pl.read_parquet(path_db)
    # .sort('item_id')
    # .select(pl.col(c).name.prefix('db_') for c in col)
    .head(10_000)
)

In [18]:
items = df_db['item_name'].unique().to_list()
items[:6]

['Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
 'Maynee Bồn ngâm chân có thể gập lại 5 lớp cách nhiệt đa chức năng du lịch di động cho',
 'SweetBunny Keo Dán Nối Mi Giả Nhanh Khô 10ml',
 'ANNIEMAL Bộ công cụ chỉnh sửa móng chân mọc ngược 10 chiếc 20mm Móng Corrector Dải nẹp đôi kết thúc tập tin có nâng lên',
 'Fred Segal quần jean nam ống rộng quần nam ống rộng quần 2023 NEW MNK2410914 2Z240111',
 'Clemine Bộ Dụng Cụ Làm Móng Tay Cầm Bằng cotton Tiện Dụng']

In [7]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [22]:
lst_pair = [(i , v) for i in items[:2] for v in items]
lst_pair[:6]

[('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé'),
 ('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'Maynee Bồn ngâm chân có thể gập lại 5 lớp cách nhiệt đa chức năng du lịch di động cho'),
 ('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'SweetBunny Keo Dán Nối Mi Giả Nhanh Khô 10ml'),
 ('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'ANNIEMAL Bộ công cụ chỉnh sửa móng chân mọc ngược 10 chiếc 20mm Móng Corrector Dải nẹp đôi kết thúc tập tin có nâng lên'),
 ('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'Fred Segal quần jean nam ống rộng quần nam ống rộng quần 2023 NEW MNK2410914 2Z240111'),
 ('Xe Cứu Hộ Đồ Chơi Cỡ Lớn Hình Khủng Long Cho Bé',
  'Clemine Bộ Dụng Cụ Làm Móng Tay Cầm Bằng cotton Tiện Dụng')]

In [23]:
score = reranker.compute_score(lst_pair, normalize=True, max_length=80)

Compute Scores: 100%|██████████| 78/78 [00:42<00:00,  1.82it/s]


In [25]:
data = (
    pl.DataFrame(lst_pair).transpose()
    .with_columns(pl.Series(values=score, name='score'))
    .sort(['score'], descending=True)
)
data

column_0,column_1,score
str,str,f64
"""Xe Cứu Hộ Đồ C…","""Xe Cứu Hộ Đồ C…",0.999979
"""Maynee Bồn ngâ…","""Maynee Bồn ngâ…",0.999979
"""Maynee Bồn ngâ…","""Texas053 Bồn n…",0.970296
"""Xe Cứu Hộ Đồ C…","""Babycare Đồ ch…",0.964987
"""Xe Cứu Hộ Đồ C…","""Haoerwu Đồ Chơ…",0.831691
…,…,…
"""Maynee Bồn ngâ…","""Paul fitzgeral…",0.000016
"""Maynee Bồn ngâ…","""Quần Jean In C…",0.000016
"""Xe Cứu Hộ Đồ C…","""Dầu Gội Sữa Dê…",0.000016
